In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Yes_Bank_Train.csv')

In [4]:
df.head()

,serial number,account_info,duration_month,credit_history,purpose,credit_amount,savings_account,employment_st,poi,personal_status,...,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
0,1,A11,6,A34,A43,1169,A65,A75,4,A93,...,4,A121,67,A143,A152,2,A173,1,A192,A201
1,2,A12,48,A32,A43,5951,A61,A73,2,A92,...,2,A121,22,A143,A152,1,A173,1,A191,A201
2,3,A14,12,A34,A46,2096,A61,A74,2,A93,...,3,A121,49,A143,A152,1,A172,2,A191,A201
3,4,A11,42,A32,A42,7882,A61,A74,2,A93,...,4,A122,45,A143,A153,1,A173,2,A191,A201
4,5,A11,24,A33,A40,4870,A61,A73,3,A93,...,4,A124,53,A143,A153,2,A173,2,A191,A201


In [5]:
df.describe()

,serial number,duration_month,credit_amount,poi,resident_since,age,credits_no,liables
count,800.0000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,400.5000,20.65125,3190.976250,2.966250,2.841250,35.406250,1.396250,1.147500
std,231.0844,12.15635,2732.671789,1.128806,1.106833,11.470317,0.569773,0.354825
min,1.0000,4.00000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,200.7500,12.00000,1342.250000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,400.5000,18.00000,2281.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,600.2500,24.00000,3914.250000,4.000000,4.000000,41.000000,2.000000,1.000000
max,800.0000,72.00000,15945.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 21 columns):
serial number       800 non-null int64
account_info        800 non-null object
duration_month      800 non-null int64
credit_history      800 non-null object
purpose             800 non-null object
credit_amount       800 non-null int64
savings_account     800 non-null object
employment_st       800 non-null object
poi                 800 non-null int64
personal_status     800 non-null object
gurantors           800 non-null object
resident_since      800 non-null int64
property_type       800 non-null object
age                 800 non-null int64
installment_type    800 non-null object
housing_type        800 non-null object
credits_no          800 non-null int64
job_type            800 non-null object
liables             800 non-null int64
telephone           800 non-null object
foreigner           800 non-null object
dtypes: int64(8), object(13)
memory usage: 131.3+ KB


In [7]:
df =df.drop(['serial number'],axis=1)

In [8]:
X = df.drop(['credit_amount'],axis=1)

In [9]:
y = df['credit_amount']

In [10]:
obj_col=X.select_dtypes(include='object')

In [11]:
obj_col = obj_col.columns.values
X = pd.get_dummies(X, columns=obj_col)

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 60 columns):
duration_month           800 non-null int64
poi                      800 non-null int64
resident_since           800 non-null int64
age                      800 non-null int64
credits_no               800 non-null int64
liables                  800 non-null int64
account_info_A11         800 non-null uint8
account_info_A12         800 non-null uint8
account_info_A13         800 non-null uint8
account_info_A14         800 non-null uint8
credit_history_A30       800 non-null uint8
credit_history_A31       800 non-null uint8
credit_history_A32       800 non-null uint8
credit_history_A33       800 non-null uint8
credit_history_A34       800 non-null uint8
purpose_A40              800 non-null uint8
purpose_A41              800 non-null uint8
purpose_A410             800 non-null uint8
purpose_A42              800 non-null uint8
purpose_A43              800 non-null uint8
purpose_A44    

In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective ='reg:linear',colsample_bytree=1,
 learning_rate= 0.01,
 max_depth= 5,
 n_estimators= 500,subsample=0.8)

In [15]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 2366.499144


In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params={
    'n_estimators':[1000],
    'max_depth':np.arange(1,7,1),
    'learning_rate':np.arange(0.001,0.1,0.01),
    'colsample_bytree':np.arange(0.1,1.1,0.1),
    'subsample':np.arange(0.5,1.0,0.1)
}
xg_reg = xgb.XGBRegressor(objective ='reg:linear',eval_metric='rmse')
gridser = GridSearchCV(xg_reg,params)
gridser.fit(X,y)

KeyboardInterrupt: 

In [ ]:
gridser.best_params_

In [106]:
gridser.best_score_

0.5891560343837304

In [30]:
df2 = pd.read_csv('Yes_Bank_Test_int.csv')

In [31]:
df2 =df2.drop(['serial number'],axis=1)
X_t = df2
obj_col=X_t.select_dtypes(include='object')
obj_col = obj_col.columns.values
X_t = pd.get_dummies(X_t, columns=obj_col)

In [32]:
X_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 60 columns):
duration_month           200 non-null int64
poi                      200 non-null int64
resident_since           200 non-null int64
age                      200 non-null int64
credits_no               200 non-null int64
liables                  200 non-null int64
account_info_A11         200 non-null uint8
account_info_A12         200 non-null uint8
account_info_A13         200 non-null uint8
account_info_A14         200 non-null uint8
credit_history_A30       200 non-null uint8
credit_history_A31       200 non-null uint8
credit_history_A32       200 non-null uint8
credit_history_A33       200 non-null uint8
credit_history_A34       200 non-null uint8
purpose_A40              200 non-null uint8
purpose_A41              200 non-null uint8
purpose_A410             200 non-null uint8
purpose_A42              200 non-null uint8
purpose_A43              200 non-null uint8
purpose_A44    

In [25]:
finalpred = xg_reg.predict(X_t)

In [26]:
_ = pd.read_csv('Yes_Bank_Test_int.csv')

In [27]:
serial = _['serial number']
data = { 'serial number': serial, 'credit_amount': finalpred}
submission = pd.DataFrame(data)
submission.head()

,serial number,credit_amount
0,1,7136.666016
1,2,7504.736816
2,3,4523.073242
3,4,6792.499023
4,5,5676.485352


In [28]:
submission.to_csv('xgb5.csv',index=False)

In [19]:
import lightgbm as lgb

In [20]:
train_data=lgb.Dataset(X_train,label=y_train)

In [21]:
test_data = lgb.Dataset(X_test,label=y_test)

In [102]:
param={'objective':'regression',
       'booster':'gbdt',
       'metric': {'l2', 'l1'},
        'num_leaves': 31,
        'learning_rate': 0.015,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
       'boosting_type': 'gbdt',
 'colsample_bytree': 0.66,
 'learning_rate': 0.015,
 'num_leaves': 16,
 'objective': 'regression',
 'random_state': 123,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.7}

In [103]:
num_round = 10000
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

[1]	valid_0's l2: 9.39965e+06	valid_0's l1: 2129.88
[2]	valid_0's l2: 9.2862e+06	valid_0's l1: 2113.86
[3]	valid_0's l2: 9.17501e+06	valid_0's l1: 2098.38
[4]	valid_0's l2: 9.06776e+06	valid_0's l1: 2084.85
[5]	valid_0's l2: 8.96395e+06	valid_0's l1: 2071.86
[6]	valid_0's l2: 8.85353e+06	valid_0's l1: 2057.84
[7]	valid_0's l2: 8.74663e+06	valid_0's l1: 2044.11
[8]	valid_0's l2: 8.64342e+06	valid_0's l1: 2030.73
[9]	valid_0's l2: 8.5566e+06	valid_0's l1: 2019.15
[10]	valid_0's l2: 8.47025e+06	valid_0's l1: 2006.71
[11]	valid_0's l2: 8.38401e+06	valid_0's l1: 1994.13
[12]	valid_0's l2: 8.30252e+06	valid_0's l1: 1981.95
[13]	valid_0's l2: 8.22143e+06	valid_0's l1: 1970.3
[14]	valid_0's l2: 8.14326e+06	valid_0's l1: 1959.52
[15]	valid_0's l2: 8.06321e+06	valid_0's l1: 1947.27
[16]	valid_0's l2: 7.97773e+06	valid_0's l1: 1935.67
[17]	valid_0's l2: 7.89192e+06	valid_0's l1: 1923.41
[18]	valid_0's l2: 7.81045e+06	valid_0's l1: 1911.81
[19]	valid_0's l2: 7.73324e+06	valid_0's l1: 1900.99
[20]	

[552]	valid_0's l2: 4.67578e+06	valid_0's l1: 1417.2
[553]	valid_0's l2: 4.677e+06	valid_0's l1: 1417.37
[554]	valid_0's l2: 4.67469e+06	valid_0's l1: 1417.24
[555]	valid_0's l2: 4.67259e+06	valid_0's l1: 1417.16
[556]	valid_0's l2: 4.67339e+06	valid_0's l1: 1417.2
[557]	valid_0's l2: 4.67197e+06	valid_0's l1: 1416.94
[558]	valid_0's l2: 4.67223e+06	valid_0's l1: 1417.2
[559]	valid_0's l2: 4.67094e+06	valid_0's l1: 1417.13
[560]	valid_0's l2: 4.67153e+06	valid_0's l1: 1417.11
[561]	valid_0's l2: 4.67091e+06	valid_0's l1: 1417.51
[562]	valid_0's l2: 4.66892e+06	valid_0's l1: 1417.53
[563]	valid_0's l2: 4.66805e+06	valid_0's l1: 1418.06
[564]	valid_0's l2: 4.66618e+06	valid_0's l1: 1418.08
[565]	valid_0's l2: 4.66438e+06	valid_0's l1: 1418.1
[566]	valid_0's l2: 4.66405e+06	valid_0's l1: 1418.21
[567]	valid_0's l2: 4.66306e+06	valid_0's l1: 1418.21
[568]	valid_0's l2: 4.6626e+06	valid_0's l1: 1418.24
[569]	valid_0's l2: 4.6631e+06	valid_0's l1: 1418.41
[570]	valid_0's l2: 4.66201e+06	vali

[972]	valid_0's l2: 4.78328e+06	valid_0's l1: 1443.3
[973]	valid_0's l2: 4.78419e+06	valid_0's l1: 1443.29
[974]	valid_0's l2: 4.78283e+06	valid_0's l1: 1442.95
[975]	valid_0's l2: 4.78144e+06	valid_0's l1: 1442.81
[976]	valid_0's l2: 4.782e+06	valid_0's l1: 1442.64
[977]	valid_0's l2: 4.78246e+06	valid_0's l1: 1442.57
[978]	valid_0's l2: 4.78274e+06	valid_0's l1: 1442.43
[979]	valid_0's l2: 4.78221e+06	valid_0's l1: 1442.46
[980]	valid_0's l2: 4.78266e+06	valid_0's l1: 1442.08
[981]	valid_0's l2: 4.78408e+06	valid_0's l1: 1442.3
[982]	valid_0's l2: 4.78601e+06	valid_0's l1: 1442.74
[983]	valid_0's l2: 4.78616e+06	valid_0's l1: 1442.68
[984]	valid_0's l2: 4.78658e+06	valid_0's l1: 1442.6
[985]	valid_0's l2: 4.78834e+06	valid_0's l1: 1442.82
[986]	valid_0's l2: 4.78932e+06	valid_0's l1: 1442.6
[987]	valid_0's l2: 4.78861e+06	valid_0's l1: 1442.43
[988]	valid_0's l2: 4.78967e+06	valid_0's l1: 1442.66
[989]	valid_0's l2: 4.7902e+06	valid_0's l1: 1442.7
[990]	valid_0's l2: 4.791e+06	valid_

[1472]	valid_0's l2: 4.91735e+06	valid_0's l1: 1483.8
[1473]	valid_0's l2: 4.91761e+06	valid_0's l1: 1483.87
[1474]	valid_0's l2: 4.91671e+06	valid_0's l1: 1483.74
[1475]	valid_0's l2: 4.91615e+06	valid_0's l1: 1483.72
[1476]	valid_0's l2: 4.9182e+06	valid_0's l1: 1484.1
[1477]	valid_0's l2: 4.92e+06	valid_0's l1: 1484.34
[1478]	valid_0's l2: 4.92012e+06	valid_0's l1: 1484.33
[1479]	valid_0's l2: 4.91989e+06	valid_0's l1: 1484.25
[1480]	valid_0's l2: 4.92022e+06	valid_0's l1: 1484.09
[1481]	valid_0's l2: 4.9216e+06	valid_0's l1: 1484.45
[1482]	valid_0's l2: 4.92287e+06	valid_0's l1: 1484.86
[1483]	valid_0's l2: 4.9244e+06	valid_0's l1: 1485.13
[1484]	valid_0's l2: 4.92433e+06	valid_0's l1: 1485.14
[1485]	valid_0's l2: 4.92537e+06	valid_0's l1: 1485.26
[1486]	valid_0's l2: 4.92481e+06	valid_0's l1: 1485.1
[1487]	valid_0's l2: 4.92479e+06	valid_0's l1: 1485.12
[1488]	valid_0's l2: 4.92412e+06	valid_0's l1: 1485.06
[1489]	valid_0's l2: 4.92339e+06	valid_0's l1: 1484.89
[1490]	valid_0's l2

[1906]	valid_0's l2: 5.01714e+06	valid_0's l1: 1510.15
[1907]	valid_0's l2: 5.01832e+06	valid_0's l1: 1510.25
[1908]	valid_0's l2: 5.01834e+06	valid_0's l1: 1510.26
[1909]	valid_0's l2: 5.01923e+06	valid_0's l1: 1510.31
[1910]	valid_0's l2: 5.01965e+06	valid_0's l1: 1510.31
[1911]	valid_0's l2: 5.01943e+06	valid_0's l1: 1510.1
[1912]	valid_0's l2: 5.01922e+06	valid_0's l1: 1509.89
[1913]	valid_0's l2: 5.01927e+06	valid_0's l1: 1509.78
[1914]	valid_0's l2: 5.01902e+06	valid_0's l1: 1509.77
[1915]	valid_0's l2: 5.01882e+06	valid_0's l1: 1509.57
[1916]	valid_0's l2: 5.0192e+06	valid_0's l1: 1509.48
[1917]	valid_0's l2: 5.01956e+06	valid_0's l1: 1509.35
[1918]	valid_0's l2: 5.02e+06	valid_0's l1: 1509.37
[1919]	valid_0's l2: 5.02047e+06	valid_0's l1: 1509.28
[1920]	valid_0's l2: 5.02086e+06	valid_0's l1: 1509.2
[1921]	valid_0's l2: 5.02022e+06	valid_0's l1: 1509.04
[1922]	valid_0's l2: 5.01898e+06	valid_0's l1: 1508.83
[1923]	valid_0's l2: 5.01886e+06	valid_0's l1: 1508.85
[1924]	valid_0's

...

[8609]	valid_0's l2: 5.29593e+06	valid_0's l1: 1572.04
[8610]	valid_0's l2: 5.29592e+06	valid_0's l1: 1572.03
[8611]	valid_0's l2: 5.29596e+06	valid_0's l1: 1572.02
[8612]	valid_0's l2: 5.29595e+06	valid_0's l1: 1572.02
[8613]	valid_0's l2: 5.296e+06	valid_0's l1: 1572.02
[8614]	valid_0's l2: 5.29604e+06	valid_0's l1: 1572.01
[8615]	valid_0's l2: 5.29609e+06	valid_0's l1: 1572.01
[8616]	valid_0's l2: 5.29613e+06	valid_0's l1: 1572.04
[8617]	valid_0's l2: 5.29618e+06	valid_0's l1: 1572.07
[8618]	valid_0's l2: 5.29619e+06	valid_0's l1: 1572.09
[8619]	valid_0's l2: 5.29623e+06	valid_0's l1: 1572.11
[8620]	valid_0's l2: 5.29616e+06	valid_0's l1: 1572.12
[8621]	valid_0's l2: 5.29617e+06	valid_0's l1: 1572.12
[8622]	valid_0's l2: 5.29621e+06	valid_0's l1: 1572.13
[8623]	valid_0's l2: 5.29618e+06	valid_0's l1: 1572.12
[8624]	valid_0's l2: 5.29621e+06	valid_0's l1: 1572.13
[8625]	valid_0's l2: 5.29621e+06	valid_0's l1: 1572.15
[8626]	valid_0's l2: 5.29626e+06	valid_0's l1: 1572.15
[8627]	valid

[8933]	valid_0's l2: 5.2961e+06	valid_0's l1: 1572.7
[8934]	valid_0's l2: 5.29605e+06	valid_0's l1: 1572.7
[8935]	valid_0's l2: 5.29603e+06	valid_0's l1: 1572.7
[8936]	valid_0's l2: 5.29606e+06	valid_0's l1: 1572.72
[8937]	valid_0's l2: 5.29609e+06	valid_0's l1: 1572.73
[8938]	valid_0's l2: 5.29609e+06	valid_0's l1: 1572.74
[8939]	valid_0's l2: 5.29611e+06	valid_0's l1: 1572.75
[8940]	valid_0's l2: 5.29614e+06	valid_0's l1: 1572.78
[8941]	valid_0's l2: 5.29623e+06	valid_0's l1: 1572.79
[8942]	valid_0's l2: 5.29628e+06	valid_0's l1: 1572.79
[8943]	valid_0's l2: 5.29632e+06	valid_0's l1: 1572.78
[8944]	valid_0's l2: 5.29631e+06	valid_0's l1: 1572.77
[8945]	valid_0's l2: 5.2963e+06	valid_0's l1: 1572.76
[8946]	valid_0's l2: 5.29632e+06	valid_0's l1: 1572.78
[8947]	valid_0's l2: 5.29633e+06	valid_0's l1: 1572.79
[8948]	valid_0's l2: 5.29632e+06	valid_0's l1: 1572.78
[8949]	valid_0's l2: 5.29628e+06	valid_0's l1: 1572.77
[8950]	valid_0's l2: 5.29624e+06	valid_0's l1: 1572.77
[8951]	valid_0'

[9273]	valid_0's l2: 5.29521e+06	valid_0's l1: 1572.83
[9274]	valid_0's l2: 5.29517e+06	valid_0's l1: 1572.83
[9275]	valid_0's l2: 5.29513e+06	valid_0's l1: 1572.83
[9276]	valid_0's l2: 5.2951e+06	valid_0's l1: 1572.82
[9277]	valid_0's l2: 5.29508e+06	valid_0's l1: 1572.81
[9278]	valid_0's l2: 5.2951e+06	valid_0's l1: 1572.81
[9279]	valid_0's l2: 5.29516e+06	valid_0's l1: 1572.82
[9280]	valid_0's l2: 5.29518e+06	valid_0's l1: 1572.82
[9281]	valid_0's l2: 5.29518e+06	valid_0's l1: 1572.82
[9282]	valid_0's l2: 5.2952e+06	valid_0's l1: 1572.82
[9283]	valid_0's l2: 5.29522e+06	valid_0's l1: 1572.82
[9284]	valid_0's l2: 5.29523e+06	valid_0's l1: 1572.83
[9285]	valid_0's l2: 5.29524e+06	valid_0's l1: 1572.83
[9286]	valid_0's l2: 5.29529e+06	valid_0's l1: 1572.84
[9287]	valid_0's l2: 5.2953e+06	valid_0's l1: 1572.84
[9288]	valid_0's l2: 5.29537e+06	valid_0's l1: 1572.85
[9289]	valid_0's l2: 5.29543e+06	valid_0's l1: 1572.87
[9290]	valid_0's l2: 5.29545e+06	valid_0's l1: 1572.88
[9291]	valid_0

[9619]	valid_0's l2: 5.29564e+06	valid_0's l1: 1573.04
[9620]	valid_0's l2: 5.29559e+06	valid_0's l1: 1573.04
[9621]	valid_0's l2: 5.29562e+06	valid_0's l1: 1573.04
[9622]	valid_0's l2: 5.29564e+06	valid_0's l1: 1573.05
[9623]	valid_0's l2: 5.29566e+06	valid_0's l1: 1573.05
[9624]	valid_0's l2: 5.29569e+06	valid_0's l1: 1573.05
[9625]	valid_0's l2: 5.29571e+06	valid_0's l1: 1573.04
[9626]	valid_0's l2: 5.29574e+06	valid_0's l1: 1573.03
[9627]	valid_0's l2: 5.29578e+06	valid_0's l1: 1573.02
[9628]	valid_0's l2: 5.29584e+06	valid_0's l1: 1573.03
[9629]	valid_0's l2: 5.29589e+06	valid_0's l1: 1573.02
[9630]	valid_0's l2: 5.29592e+06	valid_0's l1: 1573.01
[9631]	valid_0's l2: 5.29597e+06	valid_0's l1: 1573.02
[9632]	valid_0's l2: 5.29602e+06	valid_0's l1: 1573.03
[9633]	valid_0's l2: 5.29611e+06	valid_0's l1: 1573.03
[9634]	valid_0's l2: 5.29615e+06	valid_0's l1: 1573.05
[9635]	valid_0's l2: 5.29617e+06	valid_0's l1: 1573.06
[9636]	valid_0's l2: 5.29616e+06	valid_0's l1: 1573.05
[9637]	val

[9976]	valid_0's l2: 5.29492e+06	valid_0's l1: 1572.97
[9977]	valid_0's l2: 5.29495e+06	valid_0's l1: 1572.97
[9978]	valid_0's l2: 5.29498e+06	valid_0's l1: 1572.97
[9979]	valid_0's l2: 5.29501e+06	valid_0's l1: 1572.97
[9980]	valid_0's l2: 5.29503e+06	valid_0's l1: 1572.97
[9981]	valid_0's l2: 5.29506e+06	valid_0's l1: 1572.95
[9982]	valid_0's l2: 5.29511e+06	valid_0's l1: 1572.95
[9983]	valid_0's l2: 5.2952e+06	valid_0's l1: 1572.96
[9984]	valid_0's l2: 5.29521e+06	valid_0's l1: 1572.96
[9985]	valid_0's l2: 5.29521e+06	valid_0's l1: 1572.95
[9986]	valid_0's l2: 5.29522e+06	valid_0's l1: 1572.96
[9987]	valid_0's l2: 5.29528e+06	valid_0's l1: 1572.96
[9988]	valid_0's l2: 5.29521e+06	valid_0's l1: 1572.95
[9989]	valid_0's l2: 5.29528e+06	valid_0's l1: 1572.96
[9990]	valid_0's l2: 5.2953e+06	valid_0's l1: 1572.97
[9991]	valid_0's l2: 5.29527e+06	valid_0's l1: 1572.96
[9992]	valid_0's l2: 5.2953e+06	valid_0's l1: 1572.97
[9993]	valid_0's l2: 5.29527e+06	valid_0's l1: 1572.97
[9994]	valid_

In [104]:
y_pred = bst.predict(X_t, num_iteration=bst.best_iteration)

In [105]:
_ = pd.read_csv('Yes_Bank_Test_int.csv')
serial = _['serial number']
data = { 'serial number': serial, 'credit_amount': y_pred}
submission = pd.DataFrame(data)
submission.head()

,serial number,credit_amount
0,1,1881.034044
1,2,1543.173525
2,3,2707.321383
3,4,2271.319371
4,5,1307.148104


In [65]:
submission.to_csv('lighgbm3.csv',index=False)

In [106]:
params={'objective':'regression',
       'booster':'gbdt',
       'metric': {'l2', 'l1'},
        'num_leaves': 31,
        'learning_rate': 0.01,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0}
gridParams = {
    'learning_rate': [0.005,0.01,0.015],
    'n_estimators': [40,50,60],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt','dart'],
    'objective' : ['regression'],
    'random_state' : [123], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.70,0.75,0.80,0.85],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

In [108]:
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression')

In [111]:
grid = GridSearchCV(mdl, gridParams,
                    verbose=1,
                    cv=3,n_jobs=3)

In [ ]:
grid.fit(X,y)

Fitting 3 folds for each of 4320 candidates, totalling 12960 fits


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.9s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:   47.9s
[Parallel(n_jobs=3)]: Done 444 tasks      | elapsed:  2.2min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed:  3.8min
[Parallel(n_jobs=3)]: Done 1244 tasks      | elapsed:  6.0min
[Parallel(n_jobs=3)]: Done 1794 tasks      | elapsed:  9.0min
[Parallel(n_jobs=3)]: Done 2444 tasks      | elapsed: 12.0min
[Parallel(n_jobs=3)]: Done 3194 tasks      | elapsed: 16.2min
[Parallel(n_jobs=3)]: Done 4044 tasks      | elapsed: 21.0min
[Parallel(n_jobs=3)]: Done 4994 tasks      | elapsed: 26.0min
[Parallel(n_jobs=3)]: Done 6044 tasks      | elapsed: 31.6min
[Parallel(n_jobs=3)]: Done 7194 tasks      | elapsed: 37.5min
[Parallel(n_jobs=3)]: Done 8444 tasks      | elapsed: 44.5min
[Parallel(n_jobs=3)]: Done 9794 tasks      | elapsed: 52.8min
[Parallel(n_jobs=3)]: Done 11244 tasks      | elapsed: 223.6min


In [100]:
grid.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.66,
 'learning_rate': 0.015,
 'n_estimators': 60,
 'num_leaves': 16,
 'objective': 'regression',
 'random_state': 123,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.7}

In [101]:
grid.best_score_

0.3781971803689471